<a href="https://colab.research.google.com/github/aplneto/IF998/blob/main/04_-_Programa%C3%A7%C3%A3o_Din%C3%A2mica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Programação Dinâmica

In [1]:
fibonacci = lambda x: 1 if (x <= 1) else fibonacci(x-1)+fibonacci(x-2)

print(fibonacci(5))

8


In [2]:
def fib_mem(x):
  f = [1, 1]
  for i in range(2, x+1):
    f.append(f[i-1]+f[i-2])
  return f[x]

print(fib_mem(5))

8


## No contexto de MDP

* O processo de decisão de Markov satisfaz as condições:
  * A equação de Bellman fornece uma decomposição recursiva:<br>
  $
  v_{k+1}(s)\doteq\mathbb{E}\left[R_{t+1}+\gamma v_k(S_{t+1})\middle|S_t=s\right]
  $
  * A função de valor armazena soluções para reutilização (memória cache)

## Policy Evaluation
* Problema: Avaliar uma politica $\pi(a|s)$
* Solução: Utilizar a equação de Bellman
  * Evolução: $v_1\to v_2\to ... \to v_k$
  * Para cada iteração $k+1$
  * Para todo $s \in S$
  * Atualiza $v_{k+1}(s)$ a partir de $v_k(s\prime)$<br>
  $s\prime$ sendo o sucessor de $s$ de acordo com a política $\pi$

* Avaliar $v_\pi(s)$ para a política aleatória neste ambiente:

\begin{array}{|c|c|c|c|}\hline\
&1&2&3\\\hline
4&5&6&7\\\hline
8&9&10&11\\\hline
12&13&14&\hline\\\hline
\end{array}

$\pi(a|s)=0.25\space\left(\uparrow\downarrow\leftarrow\rightarrow\right)$<br>
$R-t=-1\space\text{em todas as transições.}$<br>
$R_t=0\space\text{em ambos os estados terminais, impedindo o agente de se mover}$

In [3]:
import numpy

pi = numpy.asarray([0.25] * 4)

# Recompensas
r = -1 * numpy.ones((4, 4))
# Estados terminais
r[0, 0] = 0
r[-1, -1] = 0

In [4]:
def mover(pos, d):
  '''
  Retorna a nova posição do agente a partir de sua posição anterior e da direção
  do movimento
    Arguments:
      pos (int, int): posição atual do agente
      d (str or int): direção do movimento seguindo as regras
        - n ou 0: Norte \u2191
        - s ou 1: Sul \u2193
        - l ou 2: Leste \u2192
        - o ou 3: Oeste \u2190
    Returns:
      nova_pos (int, int): nova posição do agente
  '''
  global r
  if r[pos] == 0:
    return pos
  if isinstance(d, str):
    d = 'nslo'.index(d.lower())
  funcao_movimento = (
      lambda row, column: (row-1, column), # Norte
      lambda row, column: (row+1, column), # Sul
      lambda row, column: (row, column+1), # Leste
      lambda row, column: (row, column-1) # Oeste
  )[d]
  valida = (
      lambda pos: not((pos[0] < 0 or pos[0] > 3) or (pos[1] < 0 or pos[1] > 3))
  )
  nova_pos = funcao_movimento(*pos)
  return nova_pos if valida(nova_pos) else pos

$
v_{k+1}(s) = \mathbb{E}_\pi[R_{t+1}+\gamma v_k(S_{t+1}) | St = s]\\
\quad\qquad= \sum_a\pi(a|s)\sum_{s\prime,r}p(s\prime,r|s,a)\left[r+\gamma v_k(s\prime)\right],\quad (4.5)
$

In [5]:
v = numpy.zeros((4, 4))
print("Passo 0: ")
print(numpy.round(v, 1))
gamma = 0.9
for k in range(10):
  vback = v.copy() # Tabela de backup
  for row in range(4):
    for column in range(4):
      values = vback[tuple(zip(*[mover((row, column), d) for d in 'nslo']))]
      v[row][column] = numpy.sum(
          pi * (r[row][column] + (gamma * values))
      )
  if (k in [0, 1, 2, 9]):
    print("Passo %i" % (k+1))
    print(numpy.round(v, 1))

Passo 0: 
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
Passo 1
[[ 0. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1.  0.]]
Passo 2
[[ 0.  -1.7 -1.9 -1.9]
 [-1.7 -1.9 -1.9 -1.9]
 [-1.9 -1.9 -1.9 -1.7]
 [-1.9 -1.9 -1.7  0. ]]
Passo 3
[[ 0.  -2.2 -2.7 -2.7]
 [-2.2 -2.6 -2.7 -2.7]
 [-2.7 -2.7 -2.6 -2.2]
 [-2.7 -2.7 -2.2  0. ]]
Passo 10
[[ 0.  -4.3 -5.7 -6. ]
 [-4.3 -5.3 -5.7 -5.7]
 [-5.7 -5.7 -5.3 -4.3]
 [-6.  -5.7 -4.3  0. ]]


Atualização síncrona:
* Passa por todos os estados
* Atualiza toda a tabela "simultaneamente"
* Não é _inplace_. Requer uma tabela de **backup**

### Iterative Policy Evaluation, for estimating $V \approx v_\pi$

Input $\pi$, a política a ser avaliada

Parâmetros do algoritmo: um pequeno limiar $\theta > 0$ determina a acurácia da estimativa (o mínimo necessário para a mudança da função de valor a cada passo)

Initializar $V(s)$, para todo $s\in \mathscr{S}^+$, abritrariamente, exceto para $V(\text{terminal}) = 0$

$
\text{Loop:}\\
\quad\Delta\gets0\\
\quad\text{Loop para todo } s\in \mathscr{S}:\\
\quad\quad v\gets V(s)\\
\quad\quad V(s)\gets\sum_a\pi(a|s)\sum_{s\prime,r}p(s\prime,r|s,a)[r+\gamma V(s\prime)]\\
\quad\quad\Delta\gets\max(\Delta,|v-V(s)|)\\
\text{até}\space\Delta<\theta
$

* Observe o que ocorre com a política gulosa:<br>
$\pi(s)\gets \text{argmax}_a\sum_{s\prime,r}p(s\prime,r|s,a)[r+\gamma V(s\prime)]$

In [6]:
def mostrar_politica(pi: numpy.ndarray):
  '''
  Função auxiliar para exibir a matriz de políticas com setas ao invés de
  números
    Arguments:
      pi ([4, 4, 4]): Política de ações a ser exibida
    Returns:
      friendly_pi: Matriz de política user-friendly
  '''
  # Converte o binário do vetor para inteiro
  binary = lambda arr: numpy.sum(1*(2**numpy.where(arr == 1)[0]))
  friendly_pi = []
  setas = [
      '', '\u2191', '\u2193', '\u2195', '\u2192', '\u2197', '\u2198', 'nsl',
      '\u2190', '\u2196', '\u2199', 'nso', '\u2194', 'nlo', 'slo', '\u271a'
  ]
  for row in range(4):
    friendly_pi.append([])
    for column in range(4):
      i = binary(pi[row][column])
      friendly_pi[row].append(setas[i])
  return friendly_pi

In [42]:
pi_greedy

array([[[1., 1., 1., 1.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 1., 0., 1.]],

       [[1., 0., 0., 0.],
        [1., 0., 0., 1.],
        [0., 1., 0., 1.],
        [0., 1., 0., 0.]],

       [[1., 0., 0., 0.],
        [1., 0., 1., 0.],
        [0., 1., 1., 0.],
        [0., 1., 0., 0.]],

       [[1., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [1., 1., 1., 1.]]])

In [69]:
pi_greedy = numpy.zeros((4, 4, 4))
v = numpy.zeros((4, 4))

for k in range(11):
  vback = v.copy()
  for row in range(4):
    for column in range(4):
      directions = [mover((row, column), d) for d in 'nslo']
      values = vback[tuple(zip(*directions))]
      # valor da política aleatória
      v[row][column] = numpy.sum(
          pi * (r[row][column] + (gamma * values))
      )
      action_value = (r[row][column] + (gamma * values))
      # lista as ações que maximizam a política gulosa no atual estado
      greed_actions = numpy.argwhere(action_value == numpy.amax(action_value))
      # código auxiliar para listar as funções como representações binárias
      aux = numpy.zeros((greed_actions.size, 4))
      aux[numpy.arange(greed_actions.size),greed_actions] = 1
      pi_greedy[row][column] = aux[0]
      for i in range(1, len(aux)):
        pi_greedy[row][column] = (
            numpy.logical_or(pi_greedy[row][column], aux[i])
        )
  if k in [0, 1, 2, 3, 10]:
    print("Passo %i" % k)
    politica_greedy = mostrar_politica(pi_greedy)
    for x in range(4):
      print(numpy.round(v[x], 1), politica_greedy[x])

Passo 0
[ 0. -1. -1. -1.] ['✚', '✚', '✚', '✚']
[-1. -1. -1. -1.] ['✚', '✚', '✚', '✚']
[-1. -1. -1. -1.] ['✚', '✚', '✚', '✚']
[-1. -1. -1.  0.] ['✚', '✚', '✚', '✚']
Passo 1
[ 0.  -1.7 -1.9 -1.9] ['✚', '←', '✚', '✚']
[-1.7 -1.9 -1.9 -1.9] ['↑', '✚', '✚', '✚']
[-1.9 -1.9 -1.9 -1.7] ['✚', '✚', '✚', '↓']
[-1.9 -1.9 -1.7  0. ] ['✚', '✚', '→', '✚']
Passo 2
[ 0.  -2.2 -2.7 -2.7] ['✚', '←', '←', '✚']
[-2.2 -2.6 -2.7 -2.7] ['↑', '↖', '✚', '↓']
[-2.7 -2.7 -2.6 -2.2] ['↑', '✚', '↘', '↓']
[-2.7 -2.7 -2.2  0. ] ['✚', '→', '→', '✚']
Passo 3
[ 0.  -2.7 -3.3 -3.4] ['✚', '←', '←', '↙']
[-2.7 -3.2 -3.4 -3.3] ['↑', '↑', '↙', '↓']
[-3.3 -3.4 -3.2 -2.7] ['↑', '↗', '↘', '↓']
[-3.4 -3.3 -2.7  0. ] ['↗', '→', '→', '✚']
Passo 10
[ 0.  -4.4 -5.9 -6.3] ['✚', '←', '←', '↙']
[-4.4 -5.5 -5.9 -5.9] ['↑', '↖', '↙', '↓']
[-5.9 -5.9 -5.5 -4.4] ['↑', '↗', '↘', '↓']
[-6.3 -5.9 -4.4  0. ] ['↗', '→', '→', '✚']


## Policy Iteration

* Encontrar a política ótima através de:
  * Policy Evalutaion
  * Policy Improvement $\pi\prime(s)\doteq\text{argmax}_aq_\pi(s,a)$<br><br>
  $
  \pi_0\xrightarrow[]{E}v_{\pi_0}\xrightarrow[]{I}\pi_1\xrightarrow[]{E}v_{\pi_1}\xrightarrow[]{I}\pi_2\xrightarrow[]{E}...\xrightarrow[]{I}\pi_*\xrightarrow[]{E}v_*
  $
* Se repete por K passos
* Sempre converge para a solução ótima


### Policy Iteration (usindo policy evaluation iterativa) para estimativa de $\pi\approx\pi_*$

1. Inicialização<br>
$
V(s)\in\mathbb{N}\space\text{e}\pi(s)\in\mathcal{A}(s)\space\text{definidos arbitráriamente para todo}s\in\mathcal{S}
$
1. Policy Evaluation:<br>
$
\text{Loop:}\\
\quad\Delta\gets0\\
\quad\text{Loop para cada}s\in\mathcal{S}:\\
\qquad v\gets V(s)\\
\qquad V(s)\gets\sum_{s\prime,r}p(s\prime,r|s,\pi(s))[r+\gamma V(s\prime)]\\
\qquad\Delta\gets\max(\Delta,|v-V(s)|)\\
\quad\text{até }\Delta<\theta\text{ um número real positivo e pequano que determina a acurácia da estimativa}
$
1. Policy Improvement<br>
$
\text{policy_stable}\gets\text{true}\\
\text{Para cada}s\in\mathcal{S}:\\
\quad\text{old_action}\gets\pi(s)\\
\quad\pi(s)\gets \text{argmax}_a\sum_{s\prime,r}p(s\prime,r|s,a)[r+\gamma V(s\prime)]\\
\quad\text{If old_action}\ne\pi(s)\text{, then policy_stable}\gets\text{false}
\text{If policy_stable, then stop and return}V\approx v_*\text{; else go to 2.}
$

In [98]:
V = numpy.zeros((4,4)) # Função valor
politica = numpy.random.randint(0, 3, (4, 4)) # Política aleatória
gamma = 0.9
theta = 1e-20

policy_stable = False
while not policy_stable:
  while True:
    delta = 0
    v = V.copy()
    for i in range(4):
      for j in range(4):
        # Estado futuro de acordo com a política
        future_state = mover((i,j), politica[(i,j)])
        # Atualização da função valor
        V[i][j] = 1 * (r[(i,j)] + (gamma * v[future_state]))
        delta = max(delta, abs(v[(i,j)] - V[(i,j)]))
    if delta < theta:
      break

  while not policy_stable:
    policy_stable = True
    for i in range(4):
      for j in range(4):
        old_action = politica[(i,j)]
        future_states = [mover((i, j), d) for d in 'nslo']
        # Valor dos estados futuros possíveis
        state_values = V[tuple(zip(*future_states))]
        action_values = (r[i][j] + (gamma * state_values))
        # Função valor
        politica[(i,j)] = numpy.argmax(action_values)
        if old_action != politica[(i,j)]:
          policy_stable = False

print(politica)
print(V)

[[0 3 0 0]
 [0 0 0 1]
 [0 0 2 1]
 [0 0 2 0]]
[[  0. -10. -10. -10.]
 [-10. -10. -10. -10.]
 [-10. -10. -10.  -1.]
 [-10. -10. -10.   0.]]


## Value Iteration

* Policy Iteration pode se tornar lento e custoso
  * Obs.: não é necessário convergir totalmente a função de valor para se obter melhorias significativas na política
* Value iteration: melhora a política após um único passo de Policy Evaluation
* Atualização corresponde a equação de otimização de Bellman
* Bellman Iteration: <br>
$
v_{k+1}(s) \doteq \mathbb{E}_\pi[R_{t+1}+\gamma v_k(S_{t+1})|S_t=s]\\
\qquad=\sum_a\pi(a|s)\sum_{s\prime,r}p(s\prime,r|s,a)[r+\gamma v_k(s\prime)]
$
$
v_{k+1}(s)=\max_a\mathbb{E}[R_{t+1}+\gamma v_k(S_{t+1})|S_t=s,A_t=a]\\
\qquad=\max_a\sum_{s\prime,r}p(s\prime,r|s,a)(r+\gamma v_k(s\prime))
$

### Value Iteration para estimativa de $\pi\approx\pi_*$

Parâmetros do algoritmo: um pequeno limiar $\theta > 0$ que determina a acurácia da estimativa

Inicializar $V(s)$, para todo $s\in\mathcal{S}^+$, arbitrariamente exceto em $V(\text{terminal}) = 0$.<br>
$
\text{Loop:}\\
\quad\Delta\gets0\\
\quad\text{Loop para cada}\space s\in\mathcal{S}:\\
\qquad v\gets V(s)\\
\qquad V(s)\gets\max_a\sum_{s\prime,r}p(s\prime,r|s,a)[r+\gamma V(s\prime)]\\
\qquad\Delta\gets\max(\Delta,|v-V(s)|)\\
\text{até que }\Delta<0
$

Retorna uma polítia determinística, $\pi\approx\pi_*$, tal que:<br>
$
\pi(s)=\text{argmax}_a\sum_{s\prime,r}p(s\prime,r|s,a)[r+\gamma V(s\prime)]
$

In [101]:
politica = numpy.random.randint(0,3,(4,4))
V = numpy.zeros((4, 4))

policy_stable = False
theta = 0.1
while True:
  delta = 0
  v = V.copy()
  for i in range(4):
    for j in range(4):
      directions = [mover((i, j), d) for d in 'nslo']
      # Valor dos estados futuros possíveis
      state_values = v[tuple(zip(*directions))]
      # Valor de cada ação possível
      action_value = (r[i][j] + (gamma * state_values))
      # Função valor
      V[i][j] = 1 * numpy.max(action_value)
      # atualização da política com a ação gulosa
      politica[i][j] = numpy.argmax(action_value)
      delta = max(delta, abs(v[i][j] - V[i][j]))
  if delta < theta:
    break

print(V)
print(politica)

[[ 0.   -1.   -1.9  -2.71]
 [-1.   -1.9  -2.71 -1.9 ]
 [-1.9  -2.71 -1.9  -1.  ]
 [-2.71 -1.9  -1.    0.  ]]
[[0 3 3 1]
 [0 0 0 1]
 [0 0 1 1]
 [0 2 2 0]]


In [66]:
politica

array([[0, 3, 3, 1],
       [0, 0, 0, 1],
       [0, 0, 1, 1],
       [0, 2, 2, 0]])

# PD Assíncrona

* Até então, supomos que a atualização da função valor ocorreria de forma síncrona, atalizando todos os valores simultaneamente.
* Muitas vezes isso é proibitivo devido ao grande número de estados.
* Porém, de fato isto não é necessário, é possível atualizar apenas alguns dos estados, a medida que se tem informações de novas transições.
* A convergência também pode ser demonstrada nestas situações.

<!-- resumindo: Ao invés de criar uma cópia da tabela e atualizar todos os valores numa tabela separada, é possível fazer a atualização *in-place* e ainda convergir para a solução do problema -->

## Eficiência de PD

* Tempo polinomial em |S| e |A|
* A dimensionalidade do problema aumenta exponencialmente com a quantidade de estados e variáveis de estado
* PD assíncrona é mais utilizada em problemas com muitos estados
